# This file is used for neural field simulation for SEEG stimulation,  incorporating the heterogeneity of local connectivity

In [1]:
'''
Stimulation using neural fields and forward solution
'''

from tvb.simulator.lab import *
from tvb.basic.readers import ZipReader
from tvb.datatypes.sensors import SensorsInternal
from tvb.datatypes.projections import ProjectionSurfaceSEEG

import numpy as np
import matplotlib.pyplot as plt
import os.path as op



In [3]:
pip install mne

     |████████████████████████████████| 7.4 MB 25.2 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 6.9 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
sys.path.append('/root/capsule/code/util')
from utils_simulate import CorticalSurface_new, LocalConnectivity_new, LaplaceKernel, SpatEpi
from utils_simulate import zero_columns, zero_rows
from utils_model import Spat3DEpi
from utils_integrator import HeunDeterministicAdapted
from gain_matrix_seeg import bipify_gain
#sys.path.append('/Users/dollomab/MyProjects/Epinov_trial/vep_run_Trial/fit/')
from vep_prepare_sim import read_seeg_xyz
import time

In [5]:
from utils_model4 import SpatEpiStim
from utils_simulate import CorticalSurface_new, LocalConnectivity_new, LaplaceKernel, SpatEpi, Heaviside

In [22]:
def set_up_sim_and_run(pid, electrode1, electrode2, stim_amplitude, ezlc, subject_dir, results_dir, save_res=False, plot=False):
    print('Running simulation for: ', electrode1, electrode2, ' stim amplitude: ', stim_amplitude)
    local_con_scaling = 0.9
    global_con_scaling = 0.1
    EZ = 'Left-F1-lateral-prefrontal'
    data_dir = f'{subject_dir}/tvb/'
    # Initialise a Connectivity
    print("Setup long range connectivity")
    con = connectivity.Connectivity.from_file(str(data_dir + 'connectivity.vep.zip'))
    # con.weights = np.log(con.weights+1)
    con.weights /= con.weights.max()
    con.speed = np.array([6])
    con.configure()
    roi = con.region_labels

    # Surface and local connectivity kernel
    print("Setup local connectivity")
    
    res = '_ico5'
    surf = cortex.Cortex.from_file(source_file=str(data_dir + ("/Cortex_pial" + res + ".zip")),
                                   region_mapping_file=str(data_dir + ("/Cortex_region_map" + res + ".txt")),
                                   )
    surf_region_mapping = surf.region_mapping_data.array_data
    surf.region_mapping_data.connectivity = con # now mapping has also cortical and subcortical informations

    loc_conn = LocalConnectivity_new(equation=LaplaceKernel(), cutoff=10.0)
    loc_conn.equation.parameters['b'] = 1
    loc_conn.equation.parameters['amp'] = local_con_scaling
    cort_surf = CorticalSurface_new.from_file(source_file=str(data_dir + ("/Cortex_pial" + res + ".zip")))
    cort_surf.configure()
    loc_conn.surface = cort_surf

    print("Configure cortical surface")
    surf.local_connectivity = loc_conn
    surf.configure()

    # set corpus callosum/ medial wall vertices to 0 in local connectivity
    csr_mat = surf.local_connectivity.matrix
    csr_mat = zero_columns(csr_mat, np.where(surf.region_mapping == 0)[0])
    csr_mat = zero_rows(csr_mat, np.where(surf.region_mapping == 0)[0])
    
    idx_con = np.where(roi == EZ)[0][0]  # idx in the connectivity matrix for that region
    
    
    col_index = np.where(surf.region_mapping == idx_con)[0]
    csr_mat[np.ix_(col_index,col_index)] = csr_mat[np.ix_(col_index,col_index)]*ezlc
    csr_mat.eliminate_zeros()
    
    surf.local_connectivity.matrix = csr_mat

    # Create a regionmap as will be used in the simulator later, in order to set parameters for subcortical structures
    _regmap = np.r_[surf.region_mapping, con.unmapped_indices(surf.region_mapping)]
    assert len(_regmap) == len(surf.region_mapping_data.array_data) + len(
        con.unmapped_indices(surf.region_mapping_data.array_data)) # assertion _regmap = cortical + subcortical
    idx_vertices = np.where(_regmap == idx_con)  # idx in the surface mesh vertices for that region

    # Neural mass model
    k = 0.318
    g11 = 0.5334
    g22 = 1.0
    epileptors = SpatEpiStim()
    epileptors.variables_of_interest = ('u1', 'q1', 's', 'm')
    epileptors.gamma11 = np.array([k * g11])
    epileptors.gamma12 = np.array([k * 0.1])
    epileptors.gamma22 = np.array([k * g22])
    epileptors.theta11 = np.array([-1.0])
    epileptors.theta12 = np.array([-1.0])
    epileptors.theta22 = np.array([-0.5])
    epileptors.Iext = np.array([3.1])
    # epileptors.r = np.array([0.0005])#0.0008])
    epileptors.Iext2 = np.array([0.45])
    epileptors.tau2 = np.array([10.0])
    epileptors.tt = np.array([0.17])

    epileptors.tau0 = np.array([1000.0])  # timescale of z
    epileptors.tau3 = np.array([600.0])  # timescale of m

    n_nodes = surf.vertices.shape[0] + sum(np.invert(con.cortical))  # number of vertices + subcortical regions
    epileptors.x0 = np.ones((n_nodes)) * (-3.5)
    epileptors.threshold = np.ones((n_nodes)) * (100.0)
    epileptors.Istim = np.ones((n_nodes)) * (0.)

    # Set the parameters of the model
    # Set some region to be epileptogenic zone and propagation zone
    
    epileptors.x0[idx_vertices] = -2.2  # setting this region close to criticality
    epileptors.threshold[idx_vertices] = 1.8

    # Load SEEG sensors
    seeg_names = np.genfromtxt(str(subject_dir + "/elec/seeg.xyz"), usecols=0, dtype=str)
    seeg_coord = np.genfromtxt(str(subject_dir + "/elec/seeg.xyz"), usecols=[1, 2, 3], dtype=float)
    seeg_sensors = SensorsInternal(locations=seeg_coord, labels=seeg_names)
    seeg_sensors.configure()

    # Load gain matrix and average gain across subcortical vertices,
    # because they are stil represented as a single neural mass model in TVB
    gain_type = "gain_dipole"  # "gain_inv_square" or "gain_dipole" or "gain_openmeeg_bem"
    gain_mat = np.load(str(data_dir + '/' + (gain_type + res + ".npz")))[gain_type]
    gain_region_map = np.genfromtxt(str(data_dir + ("/gain_region_map" + res + ".txt")))

    # Correcting gain matrix Unknown region such as to not take it into account
    gain_mat[:, gain_region_map == 0] = 0

    subcortial_index = np.where(con.cortical == False)[0] 
    gain_mat_new = np.zeros((seeg_sensors.number_of_sensors, len(_regmap)))
    n_cort_vert = len(surf.region_mapping_data.array_data)
    gain_mat_new[:, :n_cort_vert] = gain_mat[:, :n_cort_vert]

    # Loop across subcortical structures and average
    for rm_idx, SC_idx in enumerate(subcortial_index): #enumerate(con.unmapped_indices(surf.region_mapping)):
        gain_mat_new[:, n_cort_vert + rm_idx] = gain_mat[:, gain_region_map == SC_idx].sum(axis=1)
    # 

    '''
    # Loop across subcortical structures and average
    for rm_idx, SC_idx in enumerate(con.unmapped_indices(surf.region_mapping)):#enumerate(np.where(con.cortical == False)[0]):#enumerate(con.unmapped_indices(surf.region_mapping)):
        gain_mat_new[:, n_cort_vert + rm_idx] = gain_mat[:, gain_region_map == SC_idx].sum(axis=1)
    # np.savez_compressed(str(data_dir/"gain_tvb.npz"), gain_tvb=gain_mat_new)
    '''
    seeg_proj = ProjectionSurfaceSEEG(sensors=seeg_sensors, projection_data=gain_mat_new)

    # Stimulation
    channel = electrode1 + '-' + electrode2  # "PM'3-4"
    choi1 = np.where(seeg_names == electrode1)[0]
    choi2 = np.where(seeg_names == electrode2)[0]
    gain_signal = abs(gain_mat_new[choi2] - gain_mat_new[choi1])
    gain_signal = np.squeeze(gain_signal)

    max_roi = np.where(gain_signal == max(gain_signal))[0][0]
    # So the good region mapping to use here is _regmap, cause we now have computed subcortical regions as nodes
    # So subcortical regions can only be extracted from this _regmap since we are using gain_mat_new
    print('The label corresponding to the highest value : ', con.region_labels[_regmap[max_roi]])

    # Getting N highest values from the gain matrix
    n = 5
    n_max_roi = np.argsort(gain_signal)[-n:]
    print('The labels corresponding to the highest values : ', con.region_labels[_regmap[n_max_roi]])

    ##############
    # stimulus parameters
    freq = 50  # Hz
    T = 1. / freq * 1000.  # pulse repetition period [ms]
    tau = 2  # pulse duration [ms]
    I = stim_amplitude  # pulse intensity [mA]
    onset = 20#500  # [ms]
    stim_length = 35#3500  # [ms]
    simulation_length = 100#10000  # 13000  # [ms]
    dt = 0.1

    class vector1D(equations.DiscreteEquation):
        equation = equations.Final(default="emp")

    eqn_t = vector1D()
    parameters = {'T': T, 'tau': tau, 'amp': I, 'onset': onset}
    pulse1, _ = equations.PulseTrain(parameters=parameters).get_series_data(max_range=stim_length, step=dt)
    pulse1_ts = [p[1] for p in pulse1]
    parameters = {'T': T, 'tau': tau, 'amp': I, 'onset': onset + tau}
    pulse2, _ = equations.PulseTrain(parameters=parameters).get_series_data(max_range=stim_length, step=dt)
    pulse2_ts = [p[1] for p in pulse2]
    pulse_ts = np.asarray(pulse1_ts) - np.asarray(pulse2_ts)
    stimulus_ts = np.hstack((pulse_ts[:-1], np.zeros(int(np.ceil((simulation_length - stim_length) / dt)))))
    eqn_t.parameters['emp'] = np.copy(stimulus_ts)

    # Adding stim weights properly
    '''
    Using _regmap as region mapping for the stimuli 
    (stimulus.configure_space(region_mapping=_regmap))
    we then just need to pass the region label (ex. np.where(con.region_labels == 'Left-Amygdala'))
    to apply stimulation with a certain weight to all the vertices corresponding to this region
    (ex. stim_weights[region_label_amyg] = 1)
    This applies uniformly the same weight to all vertices corresponding to this region, cause we use
    patterns.StimuliRegion.
    Otherwise, you should use patterns.StimuliSurface.
    '''
    # for i in n_max_roi:
    #     stim_weights[_regmap[i]] = gain_signal[i]
    # stim_weights[_regmap] = gain_signal # not appropriate cause in _regmap there is a repetition of region labels
    # so in the end you assign one of the many values that the gain_signal has for that region
    stim_weights = np.zeros(n_nodes)
    for i in range(con.region_labels.size):
        stim_weights[i] = gain_signal[np.where(_regmap == i)].max()

    # NOTE TEST here just trying to stimulate on one region
    # stimreg = 'Left-CA4'
    # stimlabel = np.where(con.region_labels == stimreg)[0]
    # stim_weights[stimlabel] = 0.5
    stimulus = patterns.StimuliRegion(temporal=eqn_t,
                                      connectivity=con,
                                      weight=stim_weights)
    stimulus.configure_space(region_mapping=_regmap)
    stimulus.configure_time(np.arange(0., np.size(stimulus_ts), 1))
    ##############
    # Integrator
    heunint = HeunDeterministicAdapted(dt=dt)

    # Monitors
    mons = [monitors.TemporalAverage(period=8.),
            monitors.SpatialAverage(period=8., spatial_mask=_regmap)]
    #   monitors.iEEG(period=10., sensors=seeg_sensors, projection=seeg_proj)] TODO error this monitor missing region mapping data now for some reason

    # Global connectivity
    coupl = coupling.Difference(a=np.array([global_con_scaling]))
    # coupl = Heaviside(a=np.array([global_con_scaling]), theta=np.array([-1]))

    # Initial conditions
    # ic = [-1.46242601e+00,  -9.69344913e+00,   2.98,  0]
    ic = [-1.6242601e+00, -16.69344913e+00, 4.1, -1.11181819e+00, -9.56105974e-20, -4.38727802e-01, 0]
    ic_full = np.repeat(ic, n_nodes).reshape((len(ic), n_nodes, 1))

    # Simulator
    sim = simulator.Simulator(model=epileptors,
                              stimulus=stimulus,
                              connectivity=con,
                              coupling=coupl,
                              integrator=heunint,
                              monitors=mons,
                              surface=surf)
    sim.configure()
    sim.current_state = ic_full

    # Run
    print("Run Simulation !")
    tic = time.time()
    results = sim.run(simulation_length=simulation_length)
    print("Finished simulation.")
    print('execute for ', (time.time() - tic) / 60.0, 'mins')

    # Save results
    # (t, data_tavg), (t, data_savg), (t, data_seeg)= results
    (t, data_tavg), (t, data_savg) = results
    print(np.shape(gain_mat_new))
    if plot:
        # plot data
        srcSig = data_savg[:, 1, :, 0] - data_savg[:, 0, :, 0]
        srcSig_normal = srcSig / np.ptp(srcSig)

        # Plot raw time series
        fig = plt.figure(tight_layout=True, figsize=(15, 20))
        plt.plot(t, (srcSig - srcSig[0, :]) + np.r_[:len(roi)], 'r', linewidth=0.3)
        plt.yticks(np.arange(len(roi)), roi, fontsize=10)

        plt.xticks(fontsize=22)
        plt.ylim([-1, len(roi) + 0.5])
        plt.xlim([t[0], t[-1]])
        plt.tight_layout()
        # plt.title(f'Epileptors time series for {pid}, Ks={ks}, Kf={kf}, Kvf={kvf}, a={a}')
        plt.show()

        ####
        idx = np.where(roi == 'Left-F1-lateral-prefrontal')[0][0]
        # idx = np.where(roi == 'Left-F1-mesial-prefrontal')[0][0]
        # idx = np.where(roi == 'Right-Orbito-frontal-cortex')[0][0]
        # idx = np.where(roi == 'Right-Temporal-pole')[0][0]
        tavg = data_savg[:, :, [idx], 0]
        print(tavg.shape)
        print('Region ' + roi[idx])

        fig, axs = plt.subplots(5, 1, tight_layout=True, figsize=(9, 10))
        axs[0].set_title(f"3DEpileptor time series I={I}, T={T}, tau={tau}, r={epileptors.tau0}, ", fontsize=15)
        axs[0].plot(t[:], tavg[:, 0, :] + [0], 'C3', linewidth=0.5)
        axs[0].set_ylabel('x1')
        axs[1].plot(t[:], tavg[:, 1, :], 'C1', linewidth=0.5)
        axs[1].set_ylabel('y1')
        axs[2].plot(t[:], tavg[:, 2, :], 'C2', linewidth=0.5)
        axs[2].set_ylabel('z')
        axs[3].plot(t[:], tavg[:, 3, :], 'C4', linewidth=0.5)
        axs[3].axhline(y=1.8, linewidth=0.5)
        axs[3].set_ylabel('m')
        axs[4].plot(stimulus.time.T, stimulus.temporal_pattern.T * stimulus.weight[idx], 'C5')
        axs[4].set_ylabel('I_stim')
        # axs[5].plot(t[:], np.ones(simulation_length)*-2.2, 'C6', linewidth=0.5)
        # axs[5].set_ylabel('x0')
        for i in range(5):
            axs[i].set_xlabel('Time [ms]', fontsize=7)
        plt.show()

        ##
        '''
        reg = 'Left-F1-lateral-prefrontal'
        # reg = 'Left-F1-lateral-premotor'
        # reg = 'Left-F1-mesial-prefrontal'
        reg = 'Left-SFS-rostral'
        # idx = np.where(roi == 'Right-CA4')[0][0]
        idx = np.where(roi == reg)[0][0]
        idx = np.where(_regmap == idx)
        tavg = np.squeeze(data_tavg[:, :, [idx], 0])
        print(tavg.shape)
        # print('Region ' + roi[idx])

        fig, axs = plt.subplots(5, 1, tight_layout=True, figsize=(9, 10))
        # axs[0].set_title(f"3DEpileptor time series {reg} I={I}, T={T}, tau={tau}, r={epileptors.r}, ", fontsize=15)
        axs[0].plot(t[:], tavg[:, 0, :] - tavg[:, 1, :] + [0], 'C3', linewidth=0.5)
        axs[0].set_ylabel('x1 - x2', fontsize=20)
        axs[1].plot(t[:], tavg[:, 1, :], 'C1', linewidth=0.5)
        axs[1].set_ylabel('x2', fontsize=20)
        axs[2].plot(t[:], tavg[:, 2, :], 'C2', linewidth=0.5)
        axs[2].set_ylabel('z', fontsize=20)
        axs[3].plot(t[:], tavg[:, 3, :], 'C4', linewidth=0.5)
        axs[3].axhline(y=epileptors.threshold[idx].mean(), linewidth=0.5)
        axs[3].set_ylabel('m', fontsize=20)
        axs[4].plot(stimulus.time.T, stimulus.temporal_pattern.T * stimulus.weight[idx], 'C5')
        axs[4].set_ylabel('I_stim', fontsize=20)
        # axs[5].plot(t[:], np.ones(simulation_length)*-2.2, 'C6', linewidth=0.5)
        # axs[5].set_ylabel('x0')
        for i in range(5):
            axs[i].set_xlabel('Time [ms]', fontsize=20)
        plt.show()
        '''
    # seeg = data_seeg[:, 0, :, 0]
    # x2 - x1 timeseries
    tavg_x2x1 = data_tavg[:, 1, :, 0] - data_tavg[:, 0, :, 0]
    seeg = np.dot(gain_mat_new, tavg_x2x1.T)
    seeg_normal = seeg / np.ptp(seeg)
    seeg_normal = seeg_normal.T

    if plot:
        scaleplt = 20
        # Plot raw time series
        fig = plt.figure(tight_layout=True, figsize=(40, 70))
        plt.plot(t, (seeg_normal - seeg_normal[0, :]) * scaleplt + np.r_[:seeg_names.shape[0]] + 0.5, 'b', linewidth=1)
        plt.yticks(np.arange(seeg_names.shape[0]), seeg_names, fontsize=20)
        plt.xticks(fontsize=30)
        plt.ylim([-1, 147 + 0.5])
        plt.xlim([t[0], t[-1]])
        plt.tight_layout()
        # plt.title(f'Epileptors time series for {pid}, Ks={ks}, Kf={kf}, Kvf={kvf}, a={a}')
        plt.show()
        # plt.show(block=True)
        # plt.interactive(False)

        # averaging gain signal over all the connectivity regions
        gain_signal_savg = []
        for idx, region in enumerate(con.region_labels):
            idx_verts = np.where(_regmap == idx)
            r = gain_signal[idx_verts].max()
            # r = stimulus.spatial_pattern[idx_verts].mean()
            gain_signal_savg.append(r)
        fig = plt.figure(figsize=(20, 10))
        plt.plot(gain_signal_savg, '.')
        plt.grid()
        plt.xticks(np.r_[:con.region_labels.shape[0]], con.region_labels, rotation=90, fontsize=10)
        plt.ylabel('Gain matrix for channel ' + channel, fontsize=10)
        plt.xlabel('Region#', fontsize=12)
        fig.tight_layout()
        plt.show()

    if save_res:
        np.savez_compressed(
            f'{results_dir}/{pid}_SEEG_stim_{channel}_source_ts_glob_{global_con_scaling}_loc_{local_con_scaling}_I_{I}_lc_{ezlc}',
            time=t, tavg=data_tavg, savg=data_savg, seeg=seeg, gain=gain_mat_new, parameters=parameters,
            stim_length=stim_length, simulation_length=simulation_length,seeg_names=seeg_names,
            local_con_scaling=local_con_scaling, global_con_scaling=global_con_scaling, dt=dt, ic=ic,
            x0_array=epileptors.x0, threshold=epileptors.threshold)
        np.savez_compressed(f'{results_dir}/{pid}_SEEG_stim_{channel}_gain_signal', gain_signal=gain_signal)
    return results

In [23]:
pid = 'Patient2'

In [24]:
subject_dir = f'/data/ana/vep/'
#data_dir = f'{subject_dir}/tvb/'

In [25]:
# Stimulation
electrode1 = "PM'3"
electrode2 = "PM'4"
stim_amplitude = 3
ezlc = 5 # if ezlc=1, it equals sim_SEEG_nf.ipynb

results_dir = f'/results/'
start_time = time.time()
results = set_up_sim_and_run(pid, electrode1, electrode2, stim_amplitude, ezlc, subject_dir,
                           f'{results_dir}', save_res=False, plot=True)
print(time.time()-start_time)

Running simulation for:  PM'3 PM'4  stim amplitude:  3
Setup long range connectivity
2025-03-05 21:56:56,907 - WARNING - tvb.basic.readers - File 'average_orientations' not found in ZIP.
2025-03-05 21:56:56,908 - WARNING - tvb.basic.readers - File 'hemispheres' not found in ZIP.
2025-03-05 21:56:56,910 - WARNING - tvb.basic.readers - File 'areas' not found in ZIP.
Setup local connectivity
Configure cortical surface
The label corresponding to the highest value :  Left-F1-mesial-prefrontal
The labels corresponding to the highest values :  ['Left-F1-lateral-prefrontal' 'Left-F1-mesial-prefrontal'
 'Left-F1-mesial-prefrontal' 'Left-F1-mesial-prefrontal'
 'Left-F1-mesial-prefrontal']
2025-03-05 21:57:33,582 - WARNING - tvb.simulator.simulator - Memory estimate exceeds total available RAM.
Run Simulation !
2025-03-05 21:57:33,584 - WARNING - utils_integrator - random_state supplied for non-stochastic integration
Finished simulation.
execute for  0.42733187675476075 mins
(205, 20502)
(12, 4, 

# If ezlc=1, it equals sim_SEEG_nf.ipynb 
# Please note: Due to space limitations, I have set the simulation length to 100. In the paper, the simulation length is 10,000.
# Please reset the following variables in the function set_up_sim_and_run:
# In the paper, we set: onset = 500 # [ms]; stim_length = 3500 # [ms].
